In [1]:
import re
import sqlite3
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

## Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)
source: [Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/alcohol-recorded-per-capita-(15-)-consumption-(in-litres-of-pure-alcohol))

In [2]:
alcohol = pd.read_csv("../data/alcohol.csv")
alcohol.sample(5)

/tmp/ipykernel_10936/1078473910.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  alcohol = pd.read_csv("../data/alcohol.csv")


,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,IsLatestYear,Dim1 type,Dim1,Dim1ValueCode,Dim2 type,Dim2,Dim2ValueCode,Dim3 type,Dim3,Dim3ValueCode,DataSourceDimValueCode,DataSource,FactValueNumericPrefix,FactValueNumeric,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
2248,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,AFR,Africa,Country,GIN,Guinea,Year,2017,False,Beverage Types,Beer,SA_BEER,NaN,NaN,NaN,NaN,NaN,NaN,GISAHGIN,"(FAO) 1961-1999; (FAO, GlobalData, Wine Instit...",NaN,0.29,NaN,NaN,0.180,NaN,0.40,0.29 [0.18 – 0.4],NaN,NaN,EN,2022-06-12T15:00:00.000Z
6873,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,SEAR,South-East Asia,Country,NPL,Nepal,Year,2012,False,Beverage Types,Beer,SA_BEER,NaN,NaN,NaN,NaN,NaN,NaN,GISAHNPL,"(FAO) 1961-1999; (FAO, GlobalData, Wine Instit...",NaN,0.13,NaN,NaN,0.079,NaN,0.18,0.13 [0.079 – 0.18],NaN,NaN,EN,2022-06-12T15:00:00.000Z
36442,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,AFR,Africa,Country,CMR,Cameroon,Year,1978,False,Beverage Types,Other alcoholic beverages,SA_OTHER_ALCOHOL,NaN,NaN,NaN,NaN,NaN,NaN,GISAHCMR,"(FAO) 1961-1999; (FAO, GlobalData, Wine Instit...",NaN,4.55,NaN,NaN,NaN,NaN,NaN,4.55,NaN,NaN,EN,2018-05-10T15:00:00.000Z
26285,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,WPR,Western Pacific,Country,AUS,Australia,Year,1991,False,Beverage Types,Spirits,SA_SPIRITS,NaN,NaN,NaN,NaN,NaN,NaN,GISAHAUS,(WDT) 1961-1989; (NDRI) 1990-1999; Australia ...,NaN,1.12,NaN,NaN,NaN,NaN,NaN,1.12,NaN,NaN,EN,2018-05-10T15:00:00.000Z
19935,SA_0000001400,"Alcohol, recorded per capita (15+) consumption...",numeric,SEAR,South-East Asia,Country,TLS,Timor-Leste,Year,1998,False,Beverage Types,Beer,SA_BEER,NaN,NaN,NaN,NaN,NaN,NaN,GISAHTLS,"(FAO) 1961-2006; (FAO, Wine Institute) 2007-2017",NaN,0.40,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,EN,2018-05-10T15:00:00.000Z


In [3]:
alcohol = alcohol[alcohol.IsLatestYear & alcohol.Dim1.str.startswith("All types")]
figure_pattern = re.compile(r"[\d.]+")

def get_value(s: str) -> float:
    if isinstance(s, (int, float)):
        return s
    elif not isinstance(s, str):
        print(type(s), s)
        return 0
    
    mo = figure_pattern.match(s)
    assert mo
    return float(mo.group(0))

alcohol["alcohol_consumption"] = alcohol.Value.map(get_value)
alcohol = alcohol[[
    "Location", "SpatialDimValueCode", "alcohol_consumption", "ParentLocation"
]].rename(columns={
    "Location": "country",
    "SpatialDimValueCode": "country_code",
    "ParentLocation": "region"
})
alcohol.sort_values(by='alcohol_consumption', ascending=False).head(10)

,country,country_code,alcohol_consumption,region
639,Cook Islands,COK,12.97,Western Pacific
638,Latvia,LVA,12.90,Europe
637,Czechia,CZE,12.73,Europe
636,Lithuania,LTU,11.93,Europe
635,Austria,AUT,11.90,Europe
634,Antigua and Barbuda,ATG,11.88,Americas
633,Estonia,EST,11.65,Europe
632,France,FRA,11.44,Europe
631,Bulgaria,BGR,11.18,Europe
630,Slovenia,SVN,11.05,Europe


## GDP per capita
source: [GDP per capita (current US$) | Data](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)

In [4]:
gdp = pd.read_csv("../data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5728786.csv", skiprows=4, index_col=[0, 1])
gdp.sample(5)

,,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,...,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
Country Name,Country Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Albania,ALB,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1033.242532,1126.683340,1281.659826,1425.124219,1846.120121,2373.581292,2673.787803,2972.743618,3595.038302,4370.539711,4114.134041,4094.349699,4437.141146,4247.631356,4413.063397,4578.633208,3952.803584,4124.055390,4531.032207,5287.660817,5396.214227,5343.037704,6377.203096,6802.804519,NaN
Dominican Republic,DOM,GDP per capita (current US$),NY.GDP.PCAP.CD,203.879751,191.907441,234.041925,258.760223,273.324720,229.559605,246.771455,251.842578,255.053503,282.68160,331.890709,362.295132,420.631472,483.346021,587.657289,704.833669,754.777456,854.992868,861.513804,977.414315,1174.693354,1284.789553,1374.468640,...,2631.743580,2845.846610,2953.247928,3085.517866,2399.498123,2468.436815,3903.815184,4080.045714,4676.079111,5053.330291,5002.153233,5509.568054,5859.381545,6049.471611,6171.295064,6533.670968,6838.936735,7191.069713,7513.497951,7947.159354,8173.344646,7167.914932,8476.749688,10120.551163,NaN
Finland,FIN,GDP per capita (current US$),NY.GDP.PCAP.CD,1179.353011,1327.427224,1411.702398,1522.319242,1707.503938,1882.086858,2010.213456,2034.189050,1907.077272,2178.03525,2465.644819,2716.190092,3177.645701,4173.173047,5297.607817,6255.544641,6739.690713,7069.105738,7628.815714,9332.245264,11223.937564,10926.817967,10938.122061,...,26186.190011,24345.914822,24967.792515,26997.752990,32927.680292,37772.178111,39054.850442,41222.602000,48476.392729,53772.794239,47481.484536,46505.303179,51148.931637,47708.061278,49892.223363,50327.240290,42801.908117,43814.026506,46412.136478,49987.626158,48629.858228,49169.719339,53489.752019,50536.624467,NaN
"Congo, Rep.",COG,GDP per capita (current US$),NY.GDP.PCAP.CD,124.782359,139.995753,149.702467,150.738168,158.135593,164.250532,177.611095,185.711648,190.898664,195.50065,196.823812,223.675316,276.493942,353.752581,370.418303,472.082070,453.069200,448.501086,502.958019,670.359915,932.508544,1065.741618,1130.258494,...,780.589705,1029.960689,859.440012,910.869711,1023.088496,1314.411280,1810.588944,2116.868944,2219.912305,2848.653160,2283.949871,2962.762480,3415.062376,3753.860925,3719.651037,3622.530074,2347.818507,1970.250199,2088.500299,2512.384167,2288.808086,1838.448124,2290.382994,2447.988989,NaN
Qatar,QAT,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2557.401693,2912.958428,3439.863814,4848.630396,13393.288463,12883.176425,15566.015979,15923.992937,16632.189003,21634.620992,28218.295108,29371.466922,24301.051849,...,20207.225589,27494.771332,25836.270793,27151.032499,31440.220712,40792.276470,52468.445648,59978.861207,64706.989903,79811.597737,60733.981700,73021.309753,92992.997131,98041.362238,97630.825515,93126.149463,66984.910200,58467.235571,59407.698050,66264.081168,62827.396954,52315.660078,66838.327641,88046.320416,NaN


In [5]:
gdp.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 67"], inplace=True)
gdp.rename(columns={c: int(c) for c in gdp.columns}, inplace=True)
gdp = gdp.stack().rename("gdp per capita")
gdp.sort_index(level=2, ascending=False, inplace=True)
gdp = gdp.to_frame().reset_index(drop=False)
gdp.head()

,Country Name,Country Code,level_2,gdp per capita
0,Zimbabwe,ZWE,2022,1266.996031
1,Zambia,ZMB,2022,1487.907764
2,"Yemen, Rep.",YEM,2022,676.928385
3,World,WLD,2022,12647.480789
4,West Bank and Gaza,PSE,2022,3789.327966


In [6]:
gdp.drop_duplicates(subset=['Country Code'], keep="first", inplace=True)
gdp = gdp.drop(columns="level_2").rename(columns={"Country Name": "country", "Country Code": "country_code"})
gdp.sample(5)

,country,country_code,gdp per capita
196,Chad,TCD,716.804381
152,Guinea,GIN,1531.656475
87,Montenegro,MNE,9893.515907
61,Portugal,PRT,24274.516582
104,Madagascar,MDG,505.035528


## Life expectancy at birth
source: [Life expectancy at birth, total (years) | Data](https://data.worldbank.org/indicator/SP.DYN.LE00.IN)

In [7]:
life = pd.read_csv("../data/API_SP.DYN.LE00.IN_DS2_en_csv_v2_5728852.csv", skiprows=4, index_col=[0, 1])
life.sample(5)

,,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,...,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
Country Name,Country Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Jordan,JOR,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,52.617000,53.671000,54.658000,55.575000,56.559000,57.415000,57.869000,59.010000,59.752000,60.444000,59.870000,61.612000,62.349000,62.908000,63.434000,63.961000,64.49000,64.98100,65.436000,65.851000,66.259000,66.714000,67.154000,...,71.694000,71.918000,72.147000,72.359000,72.573000,72.772000,72.932000,73.145000,73.373000,73.591000,73.801000,74.000000,74.189000,74.366000,74.562000,74.789000,75.011000,75.215000,75.502000,75.774000,76.044000,75.184000,74.256000,NaN,NaN
Post-demographic dividend,PST,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,68.865666,69.274127,69.238701,69.441361,69.925686,69.995427,70.218803,70.523627,70.355788,70.482248,70.747422,71.051181,71.254469,71.424942,71.787062,72.094991,72.36576,72.74985,72.892395,73.212984,73.246579,73.587552,73.919144,...,77.097305,77.349384,77.644034,77.795989,77.909451,78.339361,78.426553,78.758587,79.005590,79.151088,79.475611,79.685191,79.950385,80.083872,80.260215,80.516013,80.382025,80.539510,80.583689,80.671128,80.899083,80.004475,79.797891,NaN,NaN
Philippines,PHL,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,59.227000,59.619000,59.959000,60.388000,60.541000,60.898000,61.198000,61.447000,61.650000,61.798000,61.692000,61.988000,61.831000,61.641000,61.680000,61.704000,61.61400,62.14900,62.185000,62.349000,62.499000,62.555000,62.784000,...,69.298000,69.413000,69.614000,69.781000,69.924000,70.037000,70.240000,70.282000,70.483000,70.564000,70.633000,70.754000,70.788000,70.881000,70.835000,71.151000,71.268000,71.387000,71.516000,71.689000,71.865000,72.119000,69.266000,NaN,NaN
Liechtenstein,LIE,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79.109756,76.826829,79.275610,79.636585,79.960976,81.770732,80.668293,80.948780,81.295122,82.682927,81.500000,81.841463,81.792683,82.382927,82.260976,82.073171,82.656098,82.258537,83.746341,83.041463,84.160976,81.658537,84.402439,NaN,NaN
Sint Maarten (Dutch part),SXM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,66.246000,65.730000,66.807000,68.566000,68.604000,66.403000,68.325000,69.091000,68.795000,70.158000,71.907000,71.437000,71.736000,72.096000,72.309000,72.579000,72.70900,72.26900,71.961000,71.551000,70.918000,71.228000,72.432000,...,75.034000,74.514000,74.522000,74.858000,74.148000,74.240000,75.542000,75.926000,75.417000,75.742000,75.441000,75.132000,74.684000,74.608000,74.917000,76.773000,76.954000,76.598000,75.030000,75.131000,75.437000,74.584000,73.972000,NaN,NaN


In [8]:
life.drop(columns=["Indicator Name", "Indicator Code", "Unnamed: 67"], inplace=True)
life.rename(columns={c: int(c) for c in life.columns}, inplace=True)
life = life.stack().rename("average life expectancy at birth")
life.sort_index(level=2, ascending=False, inplace=True)
life = life.to_frame().reset_index(drop=False)
life.head()

,Country Name,Country Code,level_2,average life expectancy at birth
0,Zimbabwe,ZWE,2021,59.253000
1,Zambia,ZMB,2021,61.223000
2,"Yemen, Rep.",YEM,2021,63.753000
3,World,WLD,2021,71.327391
4,West Bank and Gaza,PSE,2021,73.473000


In [9]:
life.drop_duplicates(subset=['Country Code'], keep="first", inplace=True)
life = life.drop(columns="level_2").rename(columns={"Country Name": "country", "Country Code": "country_code"})
life.sample(5)

,country,country_code,average life expectancy at birth
94,Morocco,MAR,74.042000
224,Burundi,BDI,61.663000
192,El Salvador,SLV,70.748000
200,Dominica,DMA,72.814000
124,Least developed countries: UN classification,LDC,64.215868


## Merge & Export

In [10]:
world_stats = life.merge(alcohol.drop(columns="country"), on="country_code").merge(gdp.drop(columns="country"), on="country_code")

In [11]:
with sqlite3.connect("../data/world_stats.sqlite3") as conn:
    world_stats.to_sql("world_stats", conn, index=False, if_exists="replace")

In [12]:
world_stats.sample(5)

,country,country_code,average life expectancy at birth,alcohol_consumption,region,gdp per capita
4,"Venezuela, RB",VEN,70.554000,2.51,Americas,15975.729375
111,Hungary,HUN,74.465854,10.79,Europe,18463.208525
86,Luxembourg,LUX,82.748780,11.00,Europe,126426.089934
101,Japan,JPN,84.445610,8.36,Western Pacific,33815.317273
137,Djibouti,DJI,62.305000,0.21,Eastern Mediterranean,3136.112960


In [13]:
world_stats.region.value_counts(dropna=False)

region
Europe                   50
Africa                   45
Americas                 35
Western Pacific          23
Eastern Mediterranean    21
South-East Asia          10
Name: count, dtype: int64